In [ ]:
import pandas as pd
import urllib.parse

def fetch_field(field):
    ## Enter the URL for your endpoint here.  This is the URL you tested in the browser previously
    URL = F"https://ryya1cssfj.execute-api.eu-west-1.amazonaws.com/dev?field={ urllib.parse.quote(field)}"

    print(F"URL {URL}")
    timeseries_df = pd.read_csv(URL)
    
    # print(timeseries_df)    
    
    timeseries_df['Date'] = pd.to_datetime(timeseries_df['Date'])

    timeseries_df.set_index(timeseries_df.Date, inplace=True) 
    timeseries_df = timeseries_df.resample('D').pad()
    timeseries_df.Date = timeseries_df.index.values
    timeseries_df.drop(columns=['Date'], inplace=True)

    timeseries_df.rename(columns = {'Value':field}, inplace = True)

    return timeseries_df


In [ ]:
from functools import reduce

def fetch_series(fields):
    print(f"Fields: {fields}")
    data_frames = []
    for field in fields:
        print(f"fetch {field}")
        try:
            src = fetch_field(field=field)
            print(src)
            data_frames.append(src)
        except Exception as e:
            print(f"Failed to fetch {field} : {str(e)}")
    data_frames = reduce(lambda df1, df2: pd.merge_asof(df1, df2, on='Date'), data_frames).set_index('Date')

    # print(data_frames)
    # reverse row order
    return data_frames

In [ ]:
# series_names = ['Total unique homes for sale found:', 'Average asking price of property for sale:']
series_names = ['Properties increased in price in last 14 days:', 'Properties reduced in price in last 14 days:',
                      'Properties placed on market in last 14 days:','Total unique homes for sale found:',
                      'Average asking price of property for sale:']

series_data = fetch_series(series_names)

In [ ]:
# !pip install openpyxl
with pd.ExcelWriter('output.xlsx',

                    mode='w') as writer:  

    series_data.to_excel(writer, sheet_name='Sheet_name_3')

In [ ]:
# This will fill in any gaos in the series using the value before it
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html

series_data.fillna(method="ffill", inplace=True)


This produces a graph of our data.  It will normalise the data,  that is re-scaling the data value into the same range
see https://codesource.io/how-to-normalize-data-in-python/

In [ ]:
df_min_max_scaled = series_data.copy()
                    
for column in series_names:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())    
  
# view normalized data
# display(df_min_max_scaled)

df_min_max_scaled.plot(title='National  Home stats' , figsize=(20,10));